In [20]:
import pandas as pd

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [36]:
import duckdb

con = duckdb.connect(database=':memory:')

In [21]:
athletes = pd.read_csv('data/bios.csv')

df = athletes.copy()

In [5]:
df.head()

,Roles,Sex,Full name,Used name,Born,Died,NOC,athlete_id,Measurements,Affiliations,Nick/petnames,Title(s),Other names,Nationality,Original name,Name order
0,Competed in Olympic Games,Male,"François Joseph Marie Antoine ""Jean-François""•...",Jean-François•Blanchy,"12 December 1886 in Bordeaux, Gironde (FRA)","2 October 1960 in Saint-Jean-de-Luz, Pyrénées-...",France,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Competed in Olympic Games,Male,Arnaud Benjamin•Boetsch,Arnaud•Boetsch,"1 April 1969 in Meulan, Yvelines (FRA)",NaN,France,2,183 cm / 76 kg,"Racing Club de France, Paris (FRA)",NaN,NaN,NaN,NaN,NaN,NaN
2,Competed in Olympic Games • Administrator,Male,Jean Laurent Robert•Borotra,Jean•Borotra,"13 August 1898 in Biarritz, Pyrénées-Atlantiqu...","17 July 1994 in Arbonne, Pyrénées-Atlantiques ...",France,3,183 cm / 76 kg,"TCP, Paris (FRA)",Le Basque Bondissant (The Bounding Basque),NaN,NaN,NaN,NaN,NaN
3,Competed in Olympic Games,Male,Jacques Marie Stanislas Jean•Brugnon,Jacques•Brugnon,"11 May 1895 in Paris VIIIe, Paris (FRA)","20 March 1978 in Monaco, Monaco (MON)",France,4,168 cm / 64 kg,"Sporting club de Paris, Paris (FRA)",Toto,NaN,NaN,NaN,NaN,NaN
4,Competed in Olympic Games,Male,Henry Albert•Canet,Albert•Canet,"17 April 1878 in Wandsworth, England (GBR)","25 July 1930 in Paris VIIe, Paris (FRA)",France,5,NaN,"TCP, Paris (FRA)",NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# Remove the "•" from the Used name
df['name'] = df['Used name'].str.replace("•", " ")

# Split the Measurements column into height & weight columns
df[['height_cm', 'weight_kg']] = df['Measurements'].str.split('/', expand=True)

# Get rid of " cm" and the " kg" from our new columns
df['height_cm'] = pd.to_numeric(df['height_cm'].str.strip(' cm'), errors='coerce')
df['weight_kg'] = pd.to_numeric(df['weight_kg'].str.strip(' kg'), errors='coerce')

# Parse out dates from 'Born' and 'Died' columns
date_pattern = r'(\d+ \w+ \d{4}|\d{4})'
df['born_date'] = df['Born'].str.extract(date_pattern)
df['born_year'] = df['Born'].str.extract(r'(\d{4})')

df['born_date'] = pd.to_datetime(df['born_date'], format="mixed", errors='coerce')
df['born_year'] = pd.to_numeric(df['born_year'])

df['died_date'] = df['Died'].str.extract(date_pattern)
df['died_year'] = df['Died'].str.extract(r'(\d{4})')

df['died_date'] = pd.to_datetime(df['died_date'], format="mixed", errors='coerce')
df['died_year'] = pd.to_numeric(df['died_year'])

# Get city, region, and country from Born column
location_pattern = r'in ([\w\s()-]+), ([\w\s-]+) \((\w+)\)'
df[['born_city','born_region','born_country']] = df['Born'].str.extract(location_pattern, expand=True)

location_pattern = r'in ([\w\s()-]+), ([\w\s-]+) \((\w+)\)'
df[['died_city','died_region','died_country']] = df['Died'].str.extract(location_pattern, expand=True)

# Put snake_case on columns
df.columns = df.columns.str.lower().str.replace(' ', '_')


In [23]:
df.head()

,roles,sex,full_name,used_name,born,died,noc,athlete_id,measurements,affiliations,nick/petnames,title(s),other_names,nationality,original_name,name_order,name,height_cm,weight_kg,born_date,born_year,died_date,died_year,born_city,born_region,born_country,died_city,died_region,died_country
0,Competed in Olympic Games,Male,"François Joseph Marie Antoine ""Jean-François""•Blanchy",Jean-François•Blanchy,"12 December 1886 in Bordeaux, Gironde (FRA)","2 October 1960 in Saint-Jean-de-Luz, Pyrénées-Atlantiques (FRA)",France,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jean-François Blanchy,NaN,NaN,1886-12-12,1886.0,1960-10-02,1960.0,Bordeaux,Gironde,FRA,Saint-Jean-de-Luz,Pyrénées-Atlantiques,FRA
1,Competed in Olympic Games,Male,Arnaud Benjamin•Boetsch,Arnaud•Boetsch,"1 April 1969 in Meulan, Yvelines (FRA)",NaN,France,2,183 cm / 76 kg,"Racing Club de France, Paris (FRA)",NaN,NaN,NaN,NaN,NaN,NaN,Arnaud Boetsch,183.0,76.0,1969-04-01,1969.0,NaT,NaN,Meulan,Yvelines,FRA,NaN,NaN,NaN
2,Competed in Olympic Games • Administrator,Male,Jean Laurent Robert•Borotra,Jean•Borotra,"13 August 1898 in Biarritz, Pyrénées-Atlantiques (FRA)","17 July 1994 in Arbonne, Pyrénées-Atlantiques (FRA)",France,3,183 cm / 76 kg,"TCP, Paris (FRA)",Le Basque Bondissant (The Bounding Basque),NaN,NaN,NaN,NaN,NaN,Jean Borotra,183.0,76.0,1898-08-13,1898.0,1994-07-17,1994.0,Biarritz,Pyrénées-Atlantiques,FRA,Arbonne,Pyrénées-Atlantiques,FRA
3,Competed in Olympic Games,Male,Jacques Marie Stanislas Jean•Brugnon,Jacques•Brugnon,"11 May 1895 in Paris VIIIe, Paris (FRA)","20 March 1978 in Monaco, Monaco (MON)",France,4,168 cm / 64 kg,"Sporting club de Paris, Paris (FRA)",Toto,NaN,NaN,NaN,NaN,NaN,Jacques Brugnon,168.0,64.0,1895-05-11,1895.0,1978-03-20,1978.0,Paris VIIIe,Paris,FRA,Monaco,Monaco,MON
4,Competed in Olympic Games,Male,Henry Albert•Canet,Albert•Canet,"17 April 1878 in Wandsworth, England (GBR)","25 July 1930 in Paris VIIe, Paris (FRA)",France,5,NaN,"TCP, Paris (FRA)",NaN,NaN,NaN,NaN,NaN,NaN,Albert Canet,NaN,NaN,1878-04-17,1878.0,1930-07-25,1930.0,Wandsworth,England,GBR,Paris VIIe,Paris,FRA


In [25]:
colums_to_keep = ['athlete_id', 'name', 'roles', 'sex', 'noc', 
                  'nick/petnames', 'title(s)', 'height_cm', 'weight_kg', 
                  'born_date', 'born_year', 'born_city', 'born_region', 'born_country',
                  'died_date', 'died_year', 'died_city', 'died_region', 'died_country']
df = df[colums_to_keep]

In [26]:
df.head()

,athlete_id,name,roles,sex,noc,nick/petnames,title(s),height_cm,weight_kg,born_date,born_year,born_city,born_region,born_country,died_date,died_year,died_city,died_region,died_country
0,1,Jean-François Blanchy,Competed in Olympic Games,Male,France,NaN,NaN,NaN,NaN,1886-12-12,1886.0,Bordeaux,Gironde,FRA,1960-10-02,1960.0,Saint-Jean-de-Luz,Pyrénées-Atlantiques,FRA
1,2,Arnaud Boetsch,Competed in Olympic Games,Male,France,NaN,NaN,183.0,76.0,1969-04-01,1969.0,Meulan,Yvelines,FRA,NaT,NaN,NaN,NaN,NaN
2,3,Jean Borotra,Competed in Olympic Games • Administrator,Male,France,Le Basque Bondissant (The Bounding Basque),NaN,183.0,76.0,1898-08-13,1898.0,Biarritz,Pyrénées-Atlantiques,FRA,1994-07-17,1994.0,Arbonne,Pyrénées-Atlantiques,FRA
3,4,Jacques Brugnon,Competed in Olympic Games,Male,France,Toto,NaN,168.0,64.0,1895-05-11,1895.0,Paris VIIIe,Paris,FRA,1978-03-20,1978.0,Monaco,Monaco,MON
4,5,Albert Canet,Competed in Olympic Games,Male,France,NaN,NaN,NaN,NaN,1878-04-17,1878.0,Wandsworth,England,GBR,1930-07-25,1930.0,Paris VIIe,Paris,FRA


In [27]:
results = pd.read_csv('data/results.csv')

df = results.copy()

/tmp/ipykernel_6107/2270199188.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  results = pd.read_csv('data/results.csv')


In [48]:
query = """
select distinct Discipline 
from df
where 1=1
-- and Games not like '%Summer%'
-- and Games not like '%Winter%'
-- and Games not like '%Intercalated%'
and Games like '%Equestrian%'
-- and Games like '%1956%'
-- and Games not like '%Olympic%'
"""

con.execute(query).fetchdf()

,Discipline
0,Equestrian Dressage (Equestrian)
1,Equestrian Jumping (Equestrian)
2,Equestrian Eventing (Equestrian)


In [35]:
df.head(10)

,Games,Event,Team,Pos,Medal,As,athlete_id,NOC,Discipline,Nationality,Unnamed: 7,place,tied,year,type
0,1912 Summer Olympics,"Singles, Men (Olympic)",NaN,=17,NaN,Jean-François Blanchy,1,FRA,Tennis,NaN,NaN,17,True,1912.0,Summer
1,1912 Summer Olympics,"Doubles, Men (Olympic)",Jean Montariol,DNS,NaN,Jean-François Blanchy,1,FRA,Tennis,NaN,NaN,NaN,False,1912.0,Summer
2,1920 Summer Olympics,"Singles, Men (Olympic)",NaN,=32,NaN,Jean-François Blanchy,1,FRA,Tennis,NaN,NaN,32,True,1920.0,Summer
3,1920 Summer Olympics,"Doubles, Mixed (Olympic)",Jeanne Vaussard,=8,NaN,Jean-François Blanchy,1,FRA,Tennis,NaN,NaN,8,True,1920.0,Summer
4,1920 Summer Olympics,"Doubles, Men (Olympic)",Jacques Brugnon,4,NaN,Jean-François Blanchy,1,FRA,Tennis,NaN,NaN,4,False,1920.0,Summer
5,1996 Summer Olympics,"Singles, Men (Olympic)",NaN,=17,NaN,Arnaud Boetsch,2,FRA,Tennis,NaN,NaN,17,True,1996.0,Summer
6,1996 Summer Olympics,"Doubles, Men (Olympic)",Guillaume Raoux,=17,NaN,Arnaud Boetsch,2,FRA,Tennis,NaN,NaN,17,True,1996.0,Summer
7,1924 Summer Olympics,"Singles, Men (Olympic)",NaN,4,NaN,Jean Borotra,3,FRA,Tennis,NaN,NaN,4,False,1924.0,Summer
8,1924 Summer Olympics,"Doubles, Mixed (Olympic)",Marguerite Billout,=15,NaN,Jean Borotra,3,FRA,Tennis,NaN,NaN,15,True,1924.0,Summer
9,1924 Summer Olympics,"Doubles, Men (Olympic)",René Lacoste,3,Bronze,Jean Borotra,3,FRA,Tennis,NaN,NaN,3,False,1924.0,Summer


In [29]:
df['place'] = df['Pos'].str.extract(r'(\d+)')

In [34]:
df[['year', 'type']] = df['Games'].str.extract(r'(\d{4}) (Summer|Winter)', expand=True)
df['year'] = pd.to_numeric(df['year'])